# Section 0 - Useful functions for your project

In [ ]:
# Import useful libraries
import numpy as np
import scipy.io
import scipy.optimize

Here-under we define the functions to:
- Encode a sentence into a binary vector
- Decode a binary vector into a sentence

In [ ]:
def encoding_bin(mess):
    # Convert each character to its ASCII value and then to binary
    xi = [format(ord(char), '08b') for char in mess]

    # Get the number of characters
    m = len(xi)

    # Initialize an empty list for the binary vector
    x = []

    # Convert each binary string to a binary vector
    for i in range(m):
        x.append([int(bit) for bit in xi[i]])

    # Convert the list to a numpy array for easier manipulation
    x = np.array(x)


    # Return the binary vector and its dimensions
    d = x.shape[1]  # Number of bits per character
    x = x.flatten() # convert into a 1-d vector
    return x, d

def decoding_bin(x, d):
    # Ensure x is a binary vector (0s and 1s)
    x = np.clip(x, 0, 1)  # Clip values to be between 0 and 1
    x = np.round(x)        # Round values to the nearest integer

    # Initialize the output array
    y = np.zeros((len(x) // d, d), dtype=int)

    k = 0
    for i in range(len(x) // d):
        for j in range(d):
            y[i, j] = int(x[k])  # Fill the binary matrix
            k += 1

    # Convert binary to decimal and then to characters
    mess = ''.join(chr(int(''.join(map(str, row)), 2)) for row in y)

    return mess, y

In [ ]:
message_in = "So happy to see you"
print("Message sent:", message_in)
binary_vector, dimensions = encoding_bin(message_in)
print("Binary Vector:\n", binary_vector)
float_vector = binary_vector.astype(np.float32)
message_decoded, binary_matrix = decoding_bin(float_vector, dimensions)
print("Decoded message:", message_decoded)

Message sent: So happy to see you
Binary Vector:
 [0 1 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0
 0 0 1 0 1 1 1 0 0 0 0 0 1 1 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1
 1 1 0 1 0 0 0 1 1 0 1 1 1 1 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 1 0 1 1 0 0 1 0
 1 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 1 0 1 1 0 1 1 1 1 0 1 1 1
 0 1 0 1]
Decoded message: So happy to see you


In [ ]:
dimensions

8

The function below simulates the effect of the noisy channel

In [ ]:
# Disrupts percenterror% of y entries randomly
def noisychannel(y, percenterror):
    m = len(y)                               # Length of the message
    K = int(np.floor(m * percenterror))      # Number of entries to corrupt
    I = np.random.permutation(m)[:K]         # Random indices to corrupt
    y_n = np.copy(y)                         # Copy of the orginal message
    vec = np.random.rand(K) * np.mean(y)
    y_n[I] = vec                             # Corruption of selected inputs
    return y_n

In [ ]:
# Try it
message_in = "A crystal clear message"
binary_vector, dimensions = encoding_bin(message_in)
percenterror = 0.05
float_vector = binary_vector.astype(np.float32)
yprime = noisychannel(float_vector, percenterror)
print("Message sent:", message_in)
message_corr_decoded, binary_matrix = decoding_bin(yprime, dimensions)
print("Decoded noisy message:", message_corr_decoded)

Message sent: A crystal clear message
Decoded noisy message: A crqstaL clear -e#sage


In [ ]:
float_vector.shape

(184,)

# Section 1 - Decode the Message from Alice

In [ ]:
# Load your mat file in Python
## Once your team is built, contact the Instructors by email to mention who is part of the group.
## You will then receive by return email your personal message from Alice to decrypt in .mat file.
## Alert: do not share it !

data = scipy.io.loadmat('/content/messageFromAlice.mat')
# data is dictionnay where
## data['A'] is the encoding matrix exchanged between Alice and Bob
## data['d'] is the dimension
## data['yprime'] is the encrypted message received from Alice

# Load the arrays
A = data['A']
yprime = data['yprime'].T
yprime = np.squeeze(yprime)
d = data['d'][0][0]

In [ ]:
def yourAlgorithm(A, yprime):
  # Size of A
  [m,n] = A.shape

  # Building the linear program here-under
  ## Hint: you have to express the initial Problem into a linear program in standard form
  ## Your turn

  # Solve it !
  ## Use:
  ##.    - scipy linear solver "linprog", attention to the method (HiGHS dual simplex solver or Primal Simplex or Interior Point Methods) used
  ##       https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html
  ##.    - or other linear solvers (for instance from "Gurobi"), but it has to be first accepted by the Instructors and second strongly explained/justified in the final report.
  ## For those interested, the most efficient method proposed in "linprog" SciPy function is called "highs", here are some references about it:
  ##.    - https://highs.dev/
  ##.    - Parallelizing the dual revised simplex method, Q. Huangfu and J. A. J. Hall, Mathematical Programming Computation, 10 (1), 119-142, 2018. DOI: 10.1007/s12532-017-0130-5

  # What is below is a dummy solution !
  xprime = np.random.rand(n)
  xprime.flatten()


  return xprime


Use your algorithm to solve

$min_{0 <= x^{'} <= 1} ||A*x^{'} - y^{'}||_1$


In [ ]:
xprime = yourAlgorithm(A, yprime)

In [ ]:
# Display the result:
d = 8    # Number of bits per character
message_decoded, binary_matrix = decoding_bin(xprime, d)
print("The recovered message is:", message_decoded)

# Section 2 - Generate and Decode your own messages

This section is dedicated to the fifth question of the project:
- Sending an encrypted message through a channel with sparse Gaussian noise...
- Encode and decode a message yourself:

In [ ]:
#  Message to send
my_mess = "Hey ! Welcome in Section 2."

# Message in binary form
binary_vector, d = encoding_bin(my_mess)
x = binary_vector.astype(np.float32)

# Length of the message
size = x.shape
n = size[0]

# Length of the message which will be sent
m = 4*n

# Encoding matrix: we take a randomly generated matrix
A = np.random.randn(m,n)

# Message you wish to send
y = A@x

# Noise added by the transmission channel
# = normal N(0,sigma) for a % input of y
percenterror = 0.1
yprime = noisychannel(y, percenterror)

Find x approximately from yprime by solving:


$min_{0 <= x^{'} <= 1} ||A*x^{'} - y^{'}||_1$

In [ ]:
xprime = yourAlgorithm(A, yprime)

In [ ]:
# Display the result:
d = 8    # Number of bits per character
message_decoded, binary_matrix = decoding_bin(xprime, d)
print("The recovered message is:", message_decoded)
print("The error is:", np.linalg.norm(x-xprime))

# Section 3 - Dikin's Method

This section is dedicated to the sixth question of the project:
- Implement the Dikin's Method and compare its results with the previous ones.

# Section 4 - Integer Programming

This section is dedicated to the seventh question of the project:
- by imposing binary variables: can you recover your message with a higher noise level?